In [2]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
 

In [3]:
# Importing required libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import gradio as gr




In [4]:
# Load your dataset with pandas (replace "your_dataset.csv" with the filename of your real dataset)
df = pd.read_csv("DailyDelhiClimateTrain.csv")

# Preprocess the data
# No need for scaling in Random Forests; but just check the missing values
if df.isnull().sum().sum() > 0:
  df = df.dropna()  # consider appropriate imputing method (mean, mode, median) instead of dropping NaN values.

# Split the data into independent (X) and dependent variables (y)
X = df[['meantemp','humidity']]
y = df[['wind_speed', 'meanpressure']]

# Split the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
df.corr()

<ipython-input-8-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,meantemp,humidity,wind_speed,meanpressure
meantemp,1.000000,-0.571951,0.306468,-0.038818
humidity,-0.571951,1.000000,-0.373972,0.001734
wind_speed,0.306468,-0.373972,1.000000,-0.020670
meanpressure,-0.038818,0.001734,-0.020670,1.000000


In [6]:

# Train the model
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

# Validate the model
y_pred = tree.predict(X_test)
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))



Root Mean Squared Error: 278.9950073578142


In [7]:
import requests
path = "https://api.thingspeak.com/channels/2215231/feeds.json?api_key=7IVL736EXRIIX1LH&results=2"
response = requests.get(path)
data = response.json()
data_dictionary = dict(data)
temp = data_dictionary["feeds"][1]["field1"]
humi = data_dictionary["feeds"][1]["field2"]

In [8]:

# Define the prediction function
def predict():
    # Use your input data
    meantemp = temp
    humidity = humi
    # Reshape input data to match the model input shape
    input_data = np.array([meantemp, humidity]).reshape(1, -1)
    # Predict output
    prediction = tree.predict(input_data)
    # Return the prediction
    return prediction[0][0], prediction[0][1]

# Define the Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=[],
    outputs=[gr.outputs.Label(label="Wind Speed"), gr.outputs.Label(label="Pressure")],
    title="Envio Predictor(takes Temperature and Humidity data and predicts windspeed and pressure)",
    interpretation="default"
)

# Launch the interface
iface.launch(share=True)

<ipython-input-8-bc2252a8518f>:17: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=[gr.outputs.Label(label="Wind Speed"), gr.outputs.Label(label="Pressure")],
<ipython-input-8-bc2252a8518f>:17: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  outputs=[gr.outputs.Label(label="Wind Speed"), gr.outputs.Label(label="Pressure")],


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3a48a39d3aa299fed3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
